## 1. Setup environment
We need to install miniconda (latest version). For this, we should put this in the terminal:

`wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.10.3-Linux-x86_64.sh`

And then:

`bash Miniconda3-py39_4.10.3-Linux-x86_64.sh`

If close and open the terminal is necessary, we will have to put then:

`source ~/.bashrc`

The next step is to clone the repository:

`git clone https://gitlab.cern.ch/l1a/l1_anomaly_ae.git`

Then, we have to located at:

`cd l1_anomaly_ae`

However, the Jennifer's job is at the OldBaseline tag of the AD GitLab repo:

`git checkout OldBaseline_KLL_MLT22`

After that, we have to set up the conda environment:

`conda env create -f l1ad.yml`

Finally, it must be activated:

`conda activate l1ad`
## 2. Train and evaluate DNN models

To run the full workflow, we have to execute in the terminal the following script:

`python end2end.py --run all --config config.yml`

* It runs the training and performance evaluation workflow.
* It takes as input the h5 files with L1 AD data (see the GitLab).
* Files location and model options are specified in `config.yml`.
* The script saves the trained model and the results from the evaluation step in the output directory specified in `config.yml`.
* To run using QKeras, we need to add `--model_quantize` in the script.
* `--run` can be set to `--train` or `--eval` if we want to run only one step.
* The training step dumps the data into a pickle file with name specified in `config.yml`, and it is saved in the same directory from which the script is run. We can skip this step with `--load_pickle`.
* The evaluation step dumps the predictions from the training into `results.h5` in the output directory specified at `config.yml`. We can skip this step with `--load_results`
## 3. Train and evaluate CNN models
* ***Prepare the data***

    `python prepare_data.py --input-file QCD_preprocessed.h5 --input-bsm BSM_preprocessed.h5 --output-file data.pickle`

    * `prepares_data.py` prepares the input data. It accepts preprocessed QCD and BSM h5 files as input, and outputs a pickle file with train/test events (split) to be used for the train and evaluation steps.
* ***Training***

    `python train.py --latent-dim 4 --output-model-h5 output_model.h5 --output-model-json output_model.json --input-data data.pickle --output-history history.h5 --batch-size 256 --n-epochs 30`

    * It requires:
        * Input of the latent dimension.
        * Filenames for the output model (h5 and json).
        * The training data pickle file (see *Prepare the data*).
        * The history, the batch size and the nomber of epochs.
    * Two model can be trained: Conv VAE or Conv AE. The model can be set with `--model type` as either `conv_vae` or `conv_ae`.
* ***Performance evaluation***
    * Prepare file with predictions for the QCD test sample and BSM samples:

        `python evaluate.py --input-h5 output_model.h5 --input-json output_model.json --input-history history.h5 --output-result result.h5 --input-file data.pickle`

    * make ROC curves, history and loss distributions:

        `python plot.py --coord cyl --model vae --loss-type mse_kl --output-dir ./ --input-dir ./ --label my_vae`

        Several loss types can be set and type of features.
